In [1]:
import polars as pl

from mfg_capabilities.config import config


pl.Config.set_tbl_rows(40)  # to set the number of rows displayed)
# pl.Config.set_tbl_cols(10) # to set the number of columns
# pl.Config.set_fmt_str_lengths(50) # to set the max string length displayed

polars.config.Config

In [2]:
KDT_PRODUCTION_PATH = config.data_dir / "from_bzo" / "kdt_production_20250618.csv"

# Read all columns as strings first to handle concatenated CSVs with extra headers
kdt_production_raw = pl.read_csv(
    KDT_PRODUCTION_PATH,
    has_header=True,
    infer_schema_length=0  # Treat all columns as strings
)

display(kdt_production_raw)
# display(kdt_downtime)

Active Line?,Area,Equipment,Line Name,Plant Name,Plant Number
str,str,str,str,str,str
"""YES""","""BUNS""","""GENERAL""","""BUN LINE""","""INDIANAPOLIS BAKERY""","""005"""
"""NO""","""PROCESSING""","""GENERAL""","""1 LB GOGLIO""","""PONTIAC FOODS""","""042"""
"""YES""","""DISTRIBUTION""","""GENERAL""","""DM1 - STICK PACK""","""KENLAKE FOODS""","""043"""
"""YES""","""DISTRIBUTION""","""GENERAL""","""EXTRACT""","""TARA FOODS""","""046"""
"""YES""","""DISTRIBUTION""","""GENERAL""","""752 SEMI-SOLID LINE""","""STATE AVENUE FOODS""","""049"""
"""YES""","""NOVELTIES""","""GENERAL""","""19 - SANDWICH 1""","""CROSSROAD FARMS DAIRY""","""052"""
"""YES""","""CULTURE""","""GENERAL""","""CUP FILLER""","""VANDERVOORT""","""053"""
"""NO""","""MUFFINS""","""GENERAL""","""FEDERAL FILLER 3""","""WESTOVER""","""054"""
"""YES""","""DAIRY FLUID""","""GENERAL""","""FEDERAL 1""","""TAMARACK FARMS DAIRY""","""055"""


In [16]:
plants = [
    "102", # Country Oven Bakery
    "049", # State Avenue Foods
    "714", # RCK Foods
]

kdt_production = (
    kdt_production_raw
    .filter(pl.col("Active Line?") == "YES")
    .filter(pl.col("Plant Number").is_in(plants))
    .group_by(
        "Plant Number",
        "Plant Name",
        # "Area",
        "Line Name"
    )
    .agg(pl.count())
    .sort(
        "Plant Number",
        "Plant Name",
        "Line Name"
    )
)

display(kdt_production)

/var/folders/jf/198d9jm50jq0drgyhvr6ll5r5r551n/T/ipykernel_4124/3028061896.py:17: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count())


Plant Number,Plant Name,Line Name,count
str,str,str,u32
"""049""","""STATE AVENUE FOODS""","""670 SPICE """,14
"""049""","""STATE AVENUE FOODS""","""750 LIQUID LINE""",43
"""049""","""STATE AVENUE FOODS""","""752 SEMI-SOLID LINE""",41
"""049""","""STATE AVENUE FOODS""","""770 PRESERVES LINE""",53
"""049""","""STATE AVENUE FOODS""","""771 BAKERY FILLING""",22
"""049""","""STATE AVENUE FOODS""","""772 VINEGAR LINE""",12
"""049""","""STATE AVENUE FOODS""","""790 RED SAUCE LINE""",62
"""049""","""STATE AVENUE FOODS""","""800 CANNED FOODS""",41
"""049""","""STATE AVENUE FOODS""","""820 BBQ/SYRUP LINE""",19
